In [2]:
import pandas as pd
import seaborn as sns
import pandas_profiling as pp
import tpot
import torch

In [51]:
x = torch.tensor([[1,2,], [3,4]])
y = torch.tensor([[87,98,], [99,100]])

#x[torch.tensor([0, 1])] = y

mask = torch.tensor([[True, False], [False, True]])

indices = (torch.tensor([0, 1]), torch.tensor([1])) 

z1 = x.masked_scatter(mask, y)
z1

tensor([[87,  2],
        [ 3, 98]])

## Load Dataset

In [2]:
dataset = pd.read_csv("dataset/data-final.csv", sep='\t')
print(list(dataset.columns))

['EXT1', 'EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9', 'EXT10', 'EST1', 'EST2', 'EST3', 'EST4', 'EST5', 'EST6', 'EST7', 'EST8', 'EST9', 'EST10', 'AGR1', 'AGR2', 'AGR3', 'AGR4', 'AGR5', 'AGR6', 'AGR7', 'AGR8', 'AGR9', 'AGR10', 'CSN1', 'CSN2', 'CSN3', 'CSN4', 'CSN5', 'CSN6', 'CSN7', 'CSN8', 'CSN9', 'CSN10', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5', 'OPN6', 'OPN7', 'OPN8', 'OPN9', 'OPN10', 'EXT1_E', 'EXT2_E', 'EXT3_E', 'EXT4_E', 'EXT5_E', 'EXT6_E', 'EXT7_E', 'EXT8_E', 'EXT9_E', 'EXT10_E', 'EST1_E', 'EST2_E', 'EST3_E', 'EST4_E', 'EST5_E', 'EST6_E', 'EST7_E', 'EST8_E', 'EST9_E', 'EST10_E', 'AGR1_E', 'AGR2_E', 'AGR3_E', 'AGR4_E', 'AGR5_E', 'AGR6_E', 'AGR7_E', 'AGR8_E', 'AGR9_E', 'AGR10_E', 'CSN1_E', 'CSN2_E', 'CSN3_E', 'CSN4_E', 'CSN5_E', 'CSN6_E', 'CSN7_E', 'CSN8_E', 'CSN9_E', 'CSN10_E', 'OPN1_E', 'OPN2_E', 'OPN3_E', 'OPN4_E', 'OPN5_E', 'OPN6_E', 'OPN7_E', 'OPN8_E', 'OPN9_E', 'OPN10_E', 'dateload', 'screenw', 'screenh', 'introelapse', 'testelapse', 'endelapse', 'IPC', 'count

## Clean the data

In [3]:
col_times = filter(lambda x: x.find("_E") != -1, list(dataset.columns))
dataset_clean = dataset.drop(col_times, axis=1)
dataset_clean = dataset_clean.drop(["screenw", "screenh", "testelapse", "endelapse", "IPC", "lat_appx_lots_of_err", "long_appx_lots_of_err"], axis=1)
dataset_clean = dataset_clean.drop(["dateload", "introelapse"], axis=1)

# Take subsample for faster calculations
dataset_clean = dataset_clean.sample(n=1_000_000)

# Group low-represented countries

In [16]:
counts = dataset_clean['country'].value_counts().to_dict()

def group_low_pop(x):
    if pd.isna(x):
        return None
    
    return x if (counts[x] > 1_000) and (x != "NONE") else "OTHER"

dataset_clean['country'] = dataset_clean['country'].apply(group_low_pop)

### Number of unique countries

In [22]:
dataset_clean['country'].value_counts(True)

US       0.538186
GB       0.065627
CA       0.060953
AU       0.049250
OTHER    0.037799
PH       0.019539
IN       0.017239
DE       0.013876
NZ       0.012794
NO       0.011240
MY       0.011183
MX       0.010973
SE       0.010345
NL       0.009615
SG       0.007562
ID       0.006390
BR       0.006147
FR       0.006064
DK       0.005440
IE       0.005339
IT       0.005238
ES       0.004919
PL       0.004580
FI       0.004286
RO       0.003795
BE       0.003764
ZA       0.003696
CO       0.003566
HK       0.003536
PK       0.003463
RU       0.003281
AR       0.003114
CH       0.003072
AE       0.003014
TR       0.002839
GR       0.002478
PT       0.002476
VN       0.002312
HR       0.002207
AT       0.002187
CL       0.002163
RS       0.002033
CZ       0.001983
TH       0.001929
JP       0.001887
PE       0.001638
KR       0.001561
HU       0.001487
IL       0.001410
KE       0.001385
CN       0.001314
BG       0.001250
VE       0.001241
EC       0.001136
LT       0.001084
SA       0

## Priors